In [1]:
import rasterio
from rasterio.mask import mask

# Paths
dem_hr_path = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned.tif'
clip_raster_path = '/Users/jpnousu/Krycklan_GIS_data/20m/C20/dem_clip.tif'
output_path = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/DEM/Krycklan_2015_DEM_0.5m/Krycklan_2015_DEM_cleaned_C20.tif'

# Open the clip raster to get its geometry
with rasterio.open(clip_raster_path) as clip_src:
    clip_bounds = clip_src.bounds
    clip_crs = clip_src.crs

# Open the high-res DEM
with rasterio.open(dem_hr_path) as dem_src:
    # Ensure the CRS matches
    if dem_src.crs != clip_crs:
        raise ValueError("CRS of DEM and clip raster do not match!")

    # Define bounding box geometry as a GeoJSON-like dict
    from shapely.geometry import box
    import geopandas as gpd

    bbox_geom = box(*clip_bounds)
    bbox_gdf = gpd.GeoDataFrame({'geometry': [bbox_geom]}, crs=clip_crs)
    geoms = [feature["geometry"] for feature in bbox_gdf.__geo_interface__["features"]]

    # Mask and crop DEM
    out_image, out_transform = mask(dem_src, geoms, crop=True)
    out_meta = dem_src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

# Save clipped and masked DEM
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)